<a id='wrang'></a>
# 1. Data Wrangling

In [1]:
from sklearn.datasets import make_classification
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import _euclidean_distances
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn  as sns
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
import statsmodels.api as sm
import pickle

%matplotlib inline
plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
#importing raw data
df2= pd.read_csv("C://Users//ahmed.saeed//Desktop//Job_Risk//Job_Risk//Profitability//df2_JOB_RISK2.csv", sep=',', low_memory=False)
df2

In [4]:
filename = 'C://Users//ahmed.saeed//Desktop//Job_Risk//Job_Risk//Profitability//rf_model_fitted_profitability.sav'
rf_model = pickle.load(open(filename, 'rb'))


In [5]:
# Introduing features and label
X = df2[['Brand', 'Unit', 'Job_type','Subject', 'Language_Pair','Start_TimeStamp',
         'Price','Deivery_TimeStamp','amount','Duration', 'PM', 'Account']]
y = np.array(df2["profitability"]).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)


# Step 3. – Making predictions with our model

In [6]:
y_pred = rf_model.predict(X_test)
y_pred

In [7]:
y_test

In [8]:
prob = rf_model.predict_proba(X_test)
prob

In [9]:
rf_model.classes_

In [10]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d').set_title('Maternal risks confusion matrix (0 = fail, 1 = success)')

print(classification_report(y_test,y_pred))
#Vertical (actual), horizontal (prediction)

In [11]:
# Categories Accuracy
a,b,c = (cm.diagonal()/cm.sum(axis=1))*100
print ("Low :", a,
       "\nNormal:", b,
       "\nHigh:", c)


In [12]:
importances = rf_model.feature_importances_
columns = X.columns
i = 0

while i < len(columns):
    print (f" the importance of feature '{columns[i]}' is {round(importances[i]*100, 2)}%.")
    i+=1

In [13]:
dfn1 = pd.DataFrame(prob, columns=["Low", "Normal", "High"])
dfn1['y_pred'] = y_pred
dfn1['y_test'] = y_test

dfn1

In [14]:
dfn1['25_pred'] = np.where((dfn1['Low'] >= 0.25) , 0, dfn1['y_pred'])

dfn1

In [15]:
dfn1.to_excel("C://Users//ahmed.saeed//Desktop//Job_Risk//Job_Risk//Profitability//RF_Profitability.xlsx", header=True)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
cm25 = confusion_matrix(y_test, dfn1['25_pred'])
sns.heatmap(cm25, annot=True, fmt='d').set_title('25 confusion matrix')

print(classification_report(y_test,y_pred))
#Vertical (actual), horizontal (prediction)

In [17]:
# Categories Accuracy
a,b,c = (cm25.diagonal()/cm25.sum(axis=1))*100
print ("Low :", a,
       "\nNormal:", b,
       "\nHigh:", c)

In [18]:
# final Prediction (low @ 25%)